In [1]:
!pip install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=94fb4f44231b3a7c120b32cde56abfb6668a6ad4d6316a3425dc44970f3f5469
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [2]:
import nltk
from nltk import word_tokenize
from nltk.collocations import *
from rake_nltk import Rake


In [5]:
#Step 1: tách các câu, các cụm từ các dấu delimiter(,.'-).
#Câu này chỉ là ví dụ, trong đó có các delimiter và stopword thêm ngẫu nhiên
#để loại bỏ sau này
full_text = 'Chàng trai 9X chứ gì con heo mặp - Quảng Trị khởi nghiệp - chứ nấm sò . Việc \'khởi nghiệp từ nấm sò\' gặp rất nhiều khó khăn, vì giá chứng khoán \"tăng cao\" nhanh, khiến việc thu mua nấm sò càng trở nên khó khăn.'
with open('vietnamese-stopwords.txt', 'r', encoding="utf8") as f:
    stopwords = f.readlines()

#chuyển file stopword thành list, các từ đã được loại bỏ \n ở cuối
stopword_list = list(map(str.rstrip,stopwords))

#kiểm tra một từ stopwords
print('chứ gì' in stopword_list)
r = Rake(punctuations='\'-.,"')
r.extract_keywords_from_text(full_text)
print(full_text)

#danh sách các câu, cụm dùng để tìm candidate keyword
r.get_ranked_phrases()


True
Chàng trai 9X chứ gì con heo mặp - Quảng Trị khởi nghiệp - chứ nấm sò . Việc 'khởi nghiệp từ nấm sò' gặp rất nhiều khó khăn, vì giá chứng khoán "tăng cao" nhanh, khiến việc thu mua nấm sò càng trở nên khó khăn.


['khiến việc thu mua nấm sò càng trở nên khó khăn',
 'chàng trai 9x chứ gì con heo mặp',
 'gặp rất nhiều khó khăn',
 'khởi nghiệp từ nấm sò',
 'chứ nấm sò',
 'quảng trị khởi nghiệp',
 'vì giá chứng khoán',
 'việc',
 'tăng cao',
 'nhanh']

In [6]:
#lưu danh sách các câu, cụm vào biến sent_list
sent_list = r.get_ranked_phrases()

In [7]:
#hàm remove stopword
def remove_stopword(text):
  bigram_measures = nltk.collocations.BigramAssocMeasures()
  #tokens = nltk.wordpunct_tokenize(text)
  #print(tokens)

  #Tokenize câu thành các từ riêng lẻ
  tokens = word_tokenize(text)
# print(tokens)

  #tìm tất cả các bigram có thể có
  #Ví dụ câu: 'tôi tên hải'
  #danh sách trả về : [['tôi','tên'],['tên','hải]]
  finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
  scored = finder.score_ngrams(bigram_measures.raw_freq)
  sorted(bigram for bigram, score in scored)

  bigram = sorted(bigram for bigram, score in scored)

  #với mỗi cặp bigram, nối chúng lại thành một từ: vd 'tôi tên'
  bigram_word = [' '.join(bigram) for bigram in bigram]

  #tìm kiếm các từ này trong stop word, thay thế các từ này với dấu *
  for bi in bigram_word:
    if bi in stopword_list:
      text = text.replace(bi,'*') 
  #tìm các token đơn là stop word, thay thế bằng dấu *
  for uni in word_tokenize(text):
    if uni in stopword_list:
      text = text.replace(uni,'*')
  #tách các cụm từ còn lại sau khi đã xác đinh các stop word
  #bằng cách tách theo dấu *
  candidate_keyword = [x for x in text.split('*') if x not in ['',' ']]
  return candidate_keyword

full_candidate_of_text = []
for sent in sent_list:
  full_candidate_of_text.extend(remove_stopword(sent))
full_candidate_of_text

[' thu mua nấm sò ',
 'chàng trai 9x ',
 ' heo mặp',
 'khởi nghiệp ',
 ' nấm sò',
 ' nấm sò',
 'quảng trị khởi nghiệp',
 ' giá chứng khoán']

In [10]:
# 2 ví dụ đơn giản hơn là tìm candidate keyword bằng việc loại bỏ stopword của 2 
# câu/cụm đơn, đơn giản hơn ví dụ trên
print(remove_stopword('khiến việc thu mua từ nấm sò chứ gì càng trở nên khó khăn'))
print(remove_stopword('đồng thời đóng vai trò như một biện pháp phòng ngừa đối với rủi ro tiền tệ và thị trường'))
print(remove_stopword('Giá chứng khoán của công ty FPT đang leo dốc rất nhanh và chưa có dấu hiệu hạ nhiệt'))

[' thu mua ', ' nấm sò ']
[' đóng vai trò ', ' biện pháp phòng ngừa ', ' rủi ro tiền tệ ', ' thị trường']
['Giá chứng khoán ', ' công ty FPT ', ' leo dốc ', ' dấu hiệu hạ nhiệt']
